In [1]:
# generate maps from pics
import glob
from PIL import Image
import os
import random
import glob

# dir with pics
target_dir = r'path\to\dir'
# dir with maps
out_dir = r'path\to\dir'

In [4]:
# kill repeat dots in map pics
def clear_map(pic):
    for i in range(pic.shape[0]):
        for j in range(pic.shape[0]):
            if pic[i, j] > 200:
                pic[i, j] = 255
            if pic[i, j] < 255 and pic[i, j] > 0:
                if random.random() >= pic[i, j]/255:
                    pic[i, j] = 0
                else:
                    pic[i, j] = 255
            if pic[i, j] != 0:
                pic[i, j] = 255
    y = 0
    while y < pic.shape[0]-1:
        x = 0
        while x < pic.shape[1]-1:
            if pic[x, y] == 0 and ((y>1 and pic[x, y-1] == 0) or (y<pic.shape[1]-2 and pic[x, y+1] == 0) or (x>1 and pic[x-1, y] == 0) or (x < pic.shape[1]-2 and pic[x+1, y] == 0)):
                pic[x, y] = 255
                x += 1
            x += 1
        y += 1
    x = 0
    while x < pic.shape[0]-1:
        y = 0
        while y < pic.shape[1]-1:
            if pic[x, y] == 0 and ((y>1 and pic[x, y-1] == 0) or (y<pic.shape[1]-2 and pic[x, y+1] == 0) or (x>1 and pic[x-1, y] == 0) or (x < pic.shape[1]-2 and pic[x+1, y] == 0)):
                pic[x, y] = 255
                y += 1
            y += 1
        x += 1
    return pic

def gen_galaxy(pic_name, pic):
    shape = (pic.shape[0], pic.shape[1])
    coordinates = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            if pic[i][j] < 100:
                coordinates.append((i, j))
    random.shuffle(coordinates)
    fname = os.path.splitext(os.path.basename(pic_name))[0]
    s = len(coordinates)
    out_name = os.path.join(out_dir, fname + '.txt')
    if os.path.exists(out_name):
        os.remove(out_name)
    with open(out_name, 'a') as f:
        f.write('#stars ' + str(s))
        f.write('\nstatic_galaxy_scenario = {\n\tname = \"' + fname + '\"\n\tpriority = 0\n\tdefault = no\n\tcolonizable_planet_odds = 1.0\n\tnum_empires = { min = 0 max = 30 }\n\tnum_empire_default = 14\n\tfallen_empire_default = 5\n\tfallen_empire_max = 5\n\tmarauder_empire_default = 3\n\tmarauder_empire_max = 3\n\tadvanced_empire_default = 7\n\tcore_radius = 0\n\trandom_hyperlanes = yes\n\tprimitive_odds = 1.0\n\tcrisis_strength = 1.0\n\tnum_wormhole_pairs = { min = 0 max = 5 }\n\tnum_wormhole_pairs_default = 1\n\tnum_gateways = { min = 0 max = 5 }\n\tnum_gateways_default = 1\n\tnum_hyperlanes = { min = 0.5 max = 5 }\n\tnum_hyperlanes_default = 0.75\n\n')
        f.write('\tsystem = {\n\t\tid = ' + str(0) + '\n\t\tposition = {\n\t\t\tx = ' + str(0) + '\n\t\t\ty = ' + str(0) + '\n\t\t}\n\t}\n')
        for i, coordinate in enumerate(coordinates, start = 1):
            x = (galaxy_size/shape[0] * (coordinate[1] - shape[0]/2) + random.uniform(-galaxy_size*0.005, galaxy_size*0.005))
            y = (galaxy_size/shape[1] * (coordinate[0] - shape[1]/2) + random.uniform(-galaxy_size*0.005, galaxy_size*0.005))
            # move away from border
            if x > 500:
                x -= 8
            elif x < -500:
                x += 8
            if y > 500:
                y -= 8
            elif y < -500:
                y += 8
            x = int(x)
            y = int(y)
            # write to file
            f.write('\tsystem = {\n\t\tid = ' + str(i) + '\n\t\tposition = {\n\t\t\tx = ' + str(x) + '\n\t\t\ty = ' + str(y) + '\n\t\t}\n\t}\n')
            # place some nebulas
            if random.randint(0, 200) == 1:
                f.write('\tnebula = {\n\t\tposition = {\n\t\t\tx = ' + str(random.randint(int(-galaxy_size/3), int(galaxy_size/3))) + '\n\t\t\ty = ' + str(random.randint(int(-galaxy_size/3), int(galaxy_size/3))) + '\n\t\t}\n\t\tradius = ' + str(random.randint(40, 100)) + '\n\t}\n')
        f.write('}')

galaxy_size = 1001 # maximum current (-500:500)
pics = []
for path, subdirs, files in os.walk(target_dir):
    for name in files:
        if any(i in name for i in ['.png', '.jpg']):
            pics.append(os.path.join(path, name))

for pic in pics:
    pic_name = pic
    pic = np.array(Image.open(pics).convert('P').resize((100, 100), Image.ANTIALIAS))
    pic = clear_map(pic)
    gen_galaxy(pic_name, pic)
print('done')

done


In [8]:
# # move similar pics to another folder
# import numpy as np
# def calc_mae(pic1, pic2):
#     mae = np.sum(np.absolute((pic1 - pic2)))
#     return mae

# in_pics = []
# for root, dirnames, filenames in os.walk(target_dir):
#     for filename in fnmatch.filter(filenames, '*.png'):
#         in_pics.append(os.path.join(root, filename))

# x = np.array([np.array(Image.open(pic).convert('P').resize((96, 96), Image.ANTIALIAS)) for pic in in_pics])
# x = x.astype(np.float32)
# x[x == 0] = 1
# x[x == 255] = 0

# for i in range(len(x)):
#     for j in range(i+1, len(x)):
#         mae = calc_mae(x[i], x[j])
#         if mae < 800:
#             print(in_pics[i], in_pics[j], mae)
#             out_dir = os.path.join(target_dir, str(i))
#             if not os.path.exists(out_dir):
#                 os.makedirs(out_dir)
#             try:
#                 os.rename(in_pics[i], os.path.join(out_dir, os.path.basename(in_pics[i])))
#                 os.rename(in_pics[j], os.path.join(out_dir, os.path.basename(in_pics[j])))
#             except Exception as e:
#                 e
# print('done')